In [58]:
"""This section creates the project folders for VRT."""
import pandas as pd
import os

# Read the excel file
df_project = pd.read_excel("D:/ERCOT/AEP Dynamic/MQT/MQT.xlsx",header=None)
project_list = []
for project in df_project.values.tolist():
    num = project[0]
    name = project[1]
    project_list.append(num+' '+name)
    #path = 'D:/ERCOT/AEP Dynamic/MQT/Undone/'+num+' '+name+'/'
    path = 'D:/DMView 3.3/CASEs/'+num+' '+name+'/'

    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(path)

In [59]:
"""This section creates the ini files for VRT."""
import os

project_path = 'D:/ERCOT/AEP Dynamic/MQT/Undone/'
project_list = os.listdir(project_path)

vrt_path = 'D:/DMView 3.3/'

for project in project_list:
# project = '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'
    scenario = ['dis','ch']
    initials = ['lag','lead']
    for bess in scenario:
        for initial in initials:
            """Create the ini file."""
            filename = vrt_path + project + '_VRT_' + bess + '_' + initial + '.ini'

            """Write the ini file."""
            with open(filename, 'w') as ini:
                ini.write('/' + project + 'for ERCOT dynamic model quality test\n')
                ini.write("/created by Arthur Li - CF Power\n\n")
                ini.write("[Build FS]\nBuild_FS_flag = 1\nPOI_PF = ")
                if initial == 'lag':
                    ini.write('0.95 /lagging \n')
                else:
                    ini.write('-0.95 /leading \n')

                ini.write('\n[Input files]\ninput_path = CASEs\\'+project+'\n')
                ini.write('unconv_casefile ='+project+ '_'+ bess+'.sav /psse v33\n model_file_lst = [\''+project+'.dyr\']/codes in v33\n')
                ini.write('\n[Tests]\nTest01_LVRT_LEGACY = [\'VOLT\',\'DATAs\\ERCOT_Legacy_LVRT.xlsx\']\n')
                ini.write('Test02_LVRT_VOLT_DIP = [\'VOLT\',\'DATAs\\ERCOT-245_2800_Piecewise_LVRT_(PV-BESS)_10secSpacing_r2.xlsx\']\n')
                ini.write('Test03_HVRT_PREFERED = [\'VOLT\',\'DATAs\\ERCOT-245_2800_HVRT.xlsx\']\n\n[Settings]\nPlot_Flag = 1')

In [61]:
"""This section runs the VRT."""
import os

os.chdir(vrt_path)

#for project in project_list:
project = '24INR0056 & 23INR0183 Pradera Solar & Pradera Storage'
for bess in scenario:
    for initial in initials:
        filename = vrt_path + project + '_VRT_' + bess + '_' + initial + '.ini'
        os.system('python dmview.py "'+filename+'"')
print('Finish the VRT of '+project+' successfully.\n')


Finish the VRT of 24INR0056 & 23INR0183 Pradera Solar & Pradera Storage successfully.



In [62]:
"""This section plots the VRT results."""
import os
import sys
import psse35
import psspy
import dyntools
import matplotlib.pyplot as plt
import glob
from docx import Document
from docx.shared import Inches
from io import BytesIO

sys_path_PSSE = r'C:\Program Files\PTI\PSSE35\35.5\PSSPY39'  # or where else you find the psspy.pyc
sys.path.append(sys_path_PSSE)

os_path_PSSE = r'C:\Program Files\PTI\PSSE35\35.5\PSSBIN'  # or where else you find the psse.exe
# sys.path.append(os_path_PSSE)
# os.environ['PATH'] += ';' + os_path_PSSE
# os.environ['PATH'] += ';' + sys_path_PSSE

# Create list with all .OUT files in directory
tests = ['TEST01_LVRT_LEGACY_VOLT','TEST02_LVRT_VOLT_DIP_VOLT','TEST03_HVRT_PREFERED_VOLT']
#tests = ['TEST00_FS_FS']

os.chdir(project_path+project+'/')
#os.chdir(project_path+'24INR0056 & 23INR0183 Pradera Solar & Pradera Storage/')

document = Document()
document.add_heading('VRT Report for'+project,0)

for bess in scenario:
    if bess == 'dis':
        subtitle1 = 'Discharging'
    else:
        subtitle1 = 'Charging'
    for initial in initials:
        if initial == 'lag':
            subtitle2 = 'lagging'
        else:
            subtitle2 = 'leading'

        name = project.upper()+'_VRT_' + bess + '_' + initial
        outpath = 'D:\DMView 3.3\RESULTs\\'+name
        outlist = []

        document.add_heading(subtitle1 +' '+subtitle2+':')

        for test in tests:
            out = glob.glob(outpath+'\\'+test+'\*.out')[0]
            outlist.append(out)

            outfile = dyntools.CHNF(out)

            short_title, chanid_dict, chandata_dict = outfile.get_data()

            # Plot VRT result
            p_num, q_num, v_num = 4, 6, 28 # Change the channel number here ************************************************************************************************
            time = chandata_dict['time']
            
            title = test+' at '+chanid_dict[p_num][5:]
            labels = ['Real Power','Reactive Power','Voltage']

            fig, (ax1, ax2) = plt.subplots(2, sharex=True)
            fig.suptitle(title)

            ax1.set_ylabel('Real and Reactive Power (pu)')
            ax1.plot(time, chandata_dict[p_num], color='tab:green',)
            ax1.plot(time, chandata_dict[q_num], color='tab:red',)
            ax1.grid(linestyle = '--')

            ax2.set_xlabel('Time (seconds)')
            ax2.set_ylabel('Voltage (pu)')
            ax2.plot(time, chandata_dict[v_num], color='tab:blue',)
            ax2.grid(linestyle = '--')

            fig.tight_layout()  # otherwise the right y-label is slightly clipped
            if test == 'TEST03_HVRT_PREFERED_VOLT':
                plt.xlim(0,20)
            elif test == 'TEST02_LVRT_VOLT_DIP_VOLT':
                plt.xlim(0,60)
            else:
                plt.xlim(0,25)
            fig.legend([ax1, ax2],loc = 'center right', labels=labels)

            memfile = BytesIO()
            plt.savefig(memfile)
            plt.close()
            document.add_picture(memfile, width=Inches(5))
            memfile.close()
            
document.save(project+'.docx')


C:\Users\Arthur Li\AppData\Local\Temp\ipykernel_27080\3968047066.py:83: UserWarning: You have mixed positional and keyword arguments, some input may be discarded.
  fig.legend([ax1, ax2],loc = 'center right', labels=labels)


In [57]:
print(project_path+project+'/')
print(project_path+'24INR0056 & 23INR0183 Pradera Solar & Pradera Storage/')

D:/ERCOT/AEP Dynamic/MQT/Undone/24INR0056 & 23INR00183 Pradera Solar & Pradera Storage/
D:/ERCOT/AEP Dynamic/MQT/Undone/24INR0056 & 23INR0183 Pradera Solar & Pradera Storage/
